In [3]:
import csv
import codecs
import re
import xml.etree.cElementTree as ET
import BeautifulSoup as soup

import cerberus

import schema

import pprint

# OpenStreetMap Data Case Study

## Map Area

### Hong Kong, China

http://www.openstreetmap.org/relation/913110#map=10/22.3526/114.1603

Downloaded File: https://s3.amazonaws.com/metro-extracts.mapzen.com/hong-kong_china.osm.bz2 from the below site:
https://mapzen.com/data/metro-extracts/

I am living in Hong Kong and therefore I want to explore OSM data in my hometown and I am more familiar with the place that I can find error easier. I would like to contribute to improve the OSM data.

reference: https://gist.github.com/carlward/54ec1c91b62a5f911c42#file-sample_project-md
The reference project from Udacity have guided me through the processes, which is very useful for starting the project.


# Problems Encountered in the Map

### Dataset : Hong Kong

* Chinese Character tag, however I decided not to deal with the unicode as it is impossible to clear the data completely, I will focus on the english tag instead. ( uncleaned)

` '\t\t<tag k="name:zh" v="\xe9\xa6\x99\xe6\xb8\xaf\xe5\x9c\x8b\xe9\x9a\x9b\xe6\xa9\x9f\xe5\xa0\xb4"/>\n' `

* unknown abbreviation   (unclean, unknown features)

 'LL': set(['LL'])

* abbreviation ( cleaned) 

```

 u'Rd': set(['1 Stewart Rd',
             '24 Tin Kwong Rd',
```
*  Included place not in Hong Kong. ( partly cleaned)(element.remove())

```
'Lu': set(['Hongbao Lu',
            'Hua Fa Bei Lu',
            'Shenyan Lu',
            'XiHuan Lu',
            桃园路 Taoyuan Lu,
            沙河西路 ShaHe Xi Lu,
            滨江东路 Binjiang Dong Lu]),
```

* irregular format of phone number (change to phone number start with +852 )

```

    ##set(['+852-2891-2231']) 
    ##set(['(852)26283888'])
    ##set(['+85221470111'])

```

In [4]:
class MyPrettyPrinter(pprint.PrettyPrinter):
    def format(self, object, context, maxlevels, level):
        if isinstance(object, unicode):
            return (object.encode('utf-8'), True, False)
        return pprint.PrettyPrinter.format(self, object, context, maxlevels, level)
##Set up a custom print function that can read chinese

In [5]:
## get a sense of the data
data='hong-kong_china.osm'
with open(data,'r') as file:
    x=0
    for line in file:
        if x<10:     
                MyPrettyPrinter().pprint(line)
                x=x+1;


"<?xml version='1.0' encoding='UTF-8'?>\n"
'<osm version="0.6" generator="osmconvert 0.7T" timestamp="2016-07-23T00:24:02Z">\n'
'\t<bounds minlat="21.591" minlon="112.78" maxlat="23.488" maxlon="115.125"/>\n'
'\t<node id="274901" lat="22.3460512" lon="114.1811521" version="7" timestamp="2014-01-08T15:00:31Z" changeset="19883770" uid="1242214" user="jc86035"/>\n'
'\t<node id="24323778" lat="22.3176192" lon="113.9359173" version="10" timestamp="2014-03-05T11:54:39Z" changeset="20928659" uid="1242214" user="jc86035"/>\n'
'\t<node id="24323788" lat="22.3127496" lon="113.9377728" version="6" timestamp="2014-03-05T11:54:39Z" changeset="20928659" uid="1242214" user="jc86035"/>\n'
'\t<node id="24323861" lat="22.2947833" lon="113.9376572" version="8" timestamp="2014-03-05T11:54:39Z" changeset="20928659" uid="1242214" user="jc86035"/>\n'
'\t<node id="24323865" lat="22.294111" lon="113.9380887" version="4" timestamp="2014-03-05T11:54:39Z" changeset="20928659" uid="1242214" user="jc86035"/>\n'
'\t

In [5]:
def count_tags(filename):
    dict={}
    for event, elem in ET.iterparse(filename):
       if elem.tag not in dict.keys():
           dict[elem.tag]=1
       else:
           dict[elem.tag]=dict[elem.tag]+1
       elem.clear() ### The course just mention iterparse can save memory , not until running a 500Mb i find that we need to include this function.
    return dict
## to see what kind of tag exist
tag_dict=count_tags(data)

In [6]:
print tag_dict
## These are the count of the tags
## At this point, at least the tags look good that every tag type is expected.
## I would like to take a look the tag inside the tag"tag". As many of the data are dominated by node without further tag
## It is not easy to spot problem by print every line and compare them.

{'node': 2574614, 'nd': 2984681, 'bounds': 1, 'member': 70957, 'tag': 868244, 'relation': 6304, 'way': 265372, 'osm': 1}


In [7]:
def count_k(filename):
    dic={}
    for event, elem in ET.iterparse(filename):
        if elem.tag in "tag":
            if elem.attrib['k']:
                if elem.attrib['k'] not in dic.keys():
                    dic[elem.attrib['k']]=1
                else:
                    dic[elem.attrib['k']]=dic[elem.attrib['k']]+1
    
        elem.clear() ### The course just mention iterparse can save memory , not until running a 500Mb i find that we need to include this function.
    return dic

In [61]:
tag_list=count_k(data)
## print tag_list       to see what kind of tag inside

In [78]:

#### Function inherit from quiz" Improving Street Name" and I have modify it to fit the project.
from collections import defaultdict
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

# UPDATE THIS VARIABLE
mapping = { "St": "Street",
            "St.": "Street",
            "Rd. ":"Road",
            "Ave":"Avenue",
            "Rd.":"Road",
           "Rd":"Road",
           'Ln':'Lane',
           'Av':'Avenue',
            }
unwant={"Lu":"Lu"}

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

def audit_phone_type(street_types, street_name):
    m = phone_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)
            

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def is_phone_name(elem):
    return (elem.attrib['k'] == "phone")

phone_type_re = re.compile(r'\(?\d{852}\)?|\+852|\+853|\(?\d{853}\)?')

def audit_street(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
        elem.clear()
    osm_file.close()
    return street_types
def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    phone_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_phone_name(tag):
                    audit_phone_type(phone_types, tag.attrib['v'])
                elif is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])     
        elem.clear()
    osm_file.close()
    return street_types,phone_types


def update_name(name, mapping):
    for map in mapping:
        if map in name:
           name = re.sub(r'\b' + map + r'\b\.?', mapping[map], name)
    # YOUR CODE HERE

    return name

In [79]:
st_types,phone_types = audit(data)


In [1]:
##MyPrettyPrinter().pprint((dict(phone_types)))
## Suppress printing for html output

## To check any problematic building name
##set(['+852-2891-2231'])
##set(['(852)26283888'])
##set(['+85221470111'])
## these are sample of phone number, 852 is the phone region code for Hong Kong, if it is included in the data
## I would like it start with"+852" . As the number of Hong Kong is 8 digit.



## Phone is different to Street name, as they should be group by the first part of number (ie +852), instead of street name
## that usually end with Street,Avenue,Road etc..

In [78]:
### Some correction has been done for regular expression since the unicode of chinese does affect the original function,.

In [121]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-



import csv
import codecs
import re
import xml.etree.cElementTree as ET

import cerberus

import schema

OSM_PATH = "sample.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    if element.tag == 'node':
        node_attribs = {}    
        tags = []
        
        for item in NODE_FIELDS:
            node_attribs[item] = element.attrib[item]

        for tag in element.iter("tag"):  
            if is_street_name(tag):
                if tag.attrib['v'] in unwant:
                    element.remove(element)
                    break
                else:
                    tag.attrib['v']=update_name(tag,mapping)
            elif  is_phone_name(tag):
                print tag.attrib['v']
                if phone_type_re.search(tag.attrib['v']):
                    tmp=re.split(r'\(?\d{852}\)?|\+852|\+853|\(?\d{853}\)?',tag.attrib['v'])
                    tag.attrib['v']='+852'+' '+tmp[1]
           
 
 
 
            match_prob = PROBLEMCHARS.search(tag.attrib['k'])
            if not match_prob:
                node_tag_dict = {} 
                node_tag_dict['id'] = element.attrib['id'] 
                node_tag_dict['value'] = tag.attrib['v']  

                m = LOWER_COLON.search(tag.attrib['k'])
                if not m:
                    node_tag_dict['type'] = 'regular'
                    node_tag_dict['key'] = tag.attrib['k']
                else:
                    chars_before_colon = re.findall('^(.+?):', tag.attrib['k'])
                    chars_after_colon = re.findall('^[a-z_]+:(.+)', tag.attrib['k'])

                    node_tag_dict['type'] = chars_before_colon[0]
                    node_tag_dict['key'] = chars_after_colon[0]
                    
            tags.append(node_tag_dict)
        
        return {'node': node_attribs, 'node_tags': tags}
    
    elif element.tag == 'way':
        way_attribs = {}
        way_nodes = []
        tags = []  
    
        for item in WAY_FIELDS:
            way_attribs[item] = element.attrib[item]
    
        for tag in element.iter("tag"):  
            if is_street_name(tag):
                if tag.attrib['v'] in unwant:
                    element.remove(element)
                    break ## since this element is not belong to Hong Kong, it should not be kept and directly jump out the loop.
                else:
                    tag.attrib['v']=update_name(tag,mapping)
            elif is_phone_name(tag):
                if phone_type_re.search(tag.attrib['v']):
                    tmp=re.split(r'\(?\d{852}\)?|\+852|\+853|\(?\d{853}\)?',tag.attrib['v'])
                    tag.attrib['v']='+852'+' '+tmp[1]
           
 
            match_prob = PROBLEMCHARS.search(tag.attrib['k'])
            if not match_prob:
                way_tag_dict = {}
                way_tag_dict['id'] = element.attrib['id'] 
                way_tag_dict['value'] = tag.attrib['v']  

                m = LOWER_COLON.search(tag.attrib['k'])
                if not m:
                    way_tag_dict['type'] = 'regular'
                    way_tag_dict['key'] = tag.attrib['k']
                else:
                    chars_before_colon = re.findall('^(.+?):', tag.attrib['k'])
                    chars_after_colon = re.findall('^[a-z_]+:(.+)', tag.attrib['k'])

                    way_tag_dict['type'] = chars_before_colon[0]
                    way_tag_dict['key'] = chars_after_colon[0]

            tags.append(way_tag_dict)
        count = 0
        for tag in element.iter("nd"):  
            way_nd_dict = {} 
            
            way_nd_dict['id'] = element.attrib['id'] 
            way_nd_dict['node_id'] = tag.attrib['ref'] 
            way_nd_dict['position'] = count  
            count += 1
            
            way_nodes.append(way_nd_dict)
    
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_strings = (
            "{0}: {1}".format(k, v if isinstance(v, str) else ", ".join(v))
            for k, v in errors.iteritems()
        )
        raise cerberus.ValidationError(
            message_string.format(field, "\n".join(error_strings))
        )


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is False:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])



# Note: Validation is ~ 10X slower. For the project consider using a small
# sample of the map when validating.
process_map(OSM_PATH, validate=True)


020-61972178
+86 755 8958-3637
+853 8791 9802
+85235807319
+853 2876 7668


In [115]:
## It seems work, no more error message.
 ##chars_after_colon = re.findall('^[a-z]+:(.+)', tag.attrib['k'])  has been modified to:
    ##  chars_after_colon = re.findall('^[a-z_]+:(.+)', tag.attrib['k'])

# SQL for this part

SyntaxError: invalid syntax (<ipython-input-122-19b23891d7c6>, line 1)

In [146]:
import sqlite3
import csv

##Connect to the database 
sqlite_file = 'my_db.db'    # name of the sqlite database file

# Connect to the database
conn = sqlite3.connect(sqlite_file)
cur = conn.cursor()

In [147]:
# Create the table, specifying the column names and data types:
cur.execute('''
    CREATE TABLE nodes_tags(id INTEGER, key TEXT, value TEXT,type TEXT)
''')
# commit the changes
conn.commit()

In [148]:
def unicode_csv_reader(unicode_csv_data, dialect=csv.excel, **kwargs):
    # csv.py doesn't do Unicode; encode temporarily as UTF-8:
    csv_reader = csv.reader(utf_8_encoder(unicode_csv_data),
                            dialect=dialect, **kwargs)
    for row in csv_reader:
        # decode UTF-8 back to Unicode, cell by cell:
        yield [unicode(cell, 'utf-8') for cell in row]

def utf_8_encoder(unicode_csv_data):
    for line in unicode_csv_data:
        yield line.encode('utf-8')

def UnicodeDictReader(utf8_data, **kwargs):
    csv_reader = csv.DictReader(utf8_data, **kwargs)
    for row in csv_reader:
        yield {key: unicode(value, 'utf-8') for key, value in row.iteritems()}
##reference : https://discussions.udacity.com/t/sqlite-csv-import-problem/171479/2
## utf_8_enconder to deal with the csv error
##ProgrammingError: You must not use 8-bit bytestrings unless you use a text_factory that can interpret 8-bit bytestrings (like text_factory = str). It is highly recommended that you instead just switch your application to Unicode strings.

In [149]:
CSVFILE = "nodes_tags.csv"

with open (CSVFILE, 'rb') as csvfile:
    # instead of using 'csv.DictReader' use `UnicodeDictReader`
    csv_reader = UnicodeDictReader(csvfile)

    to_db = [(i['id'], i['key'],i['value'], i['type']) for i in csv_reader]
    # insert the formatted data
cur.executemany("INSERT INTO nodes_tags(id, key, value,type) VALUES (?, ?, ?, ?);", to_db)
# commit the changes
conn.commit()

conn.close()

In [150]:
conn = sqlite3.connect(sqlite_file)
cur = conn.cursor()
cur.execute('''
    CREATE TABLE nodes(id INTEGER PRIMARY KEY,                  
lat FLOAT,
lon FLOAT,
user TEXT,
uid  TEXT,
version STRING,
changeset INTEGER,
timestamp STRING)
''')
# commit the changes
conn.commit()

CSVFILE = "nodes.csv"

with open (CSVFILE, 'rb') as csvfile:
    # instead of using 'csv.DictReader' use `UnicodeDictReader`
    csv_reader = UnicodeDictReader(csvfile)

    to_db = [(i['id'], i['lat'],i['lon'], i['user'],i['uid'],i['version'],i['changeset'],i['timestamp']) for i in csv_reader]
    # insert the formatted data
cur.executemany("INSERT INTO nodes(id, lat,lon,user,uid,version,changeset,timestamp) VALUES (?, ?, ?, ?, ?, ? , ? ,? );", to_db)
# commit the changes
conn.commit()

conn.close()

In [151]:
conn = sqlite3.connect(sqlite_file)
cur = conn.cursor()
cur.execute('''
   CREATE TABLE ways(
id INTEGER ,
user TEXT,
uid INTEGER,
version TEXT,
changeset INTEGER,
timestamp TEXT)
''')
# commit the changes
conn.commit()

CSVFILE = "ways.csv"

with open (CSVFILE, 'rb') as csvfile:
    # instead of using 'csv.DictReader' use `UnicodeDictReader`
    csv_reader = UnicodeDictReader(csvfile)

    to_db = [(i['id'], i['user'],i['uid'], i['version'],i['changeset'],i['timestamp']) for i in csv_reader]
    # insert the formatted data
cur.executemany("INSERT INTO ways(id, user,uid,version,changeset,timestamp) VALUES (?, ?, ?, ? ,? ,?);", to_db)
# commit the changes
conn.commit()

conn.close()

In [165]:
conn = sqlite3.connect(sqlite_file)
cur = conn.cursor()
cur.execute('''
   CREATE TABLE ways_nodes(
id INTEGER ,
node_id INTEGER,
position INTEGER)
''')
# commit the changes
conn.commit()

CSVFILE = "ways_nodes.csv"

with open (CSVFILE, 'rb') as csvfile:
    # instead of using 'csv.DictReader' use `UnicodeDictReader`
    csv_reader = UnicodeDictReader(csvfile)

    to_db = [(i['id'], i['node_id'],i['position']) for i in csv_reader]
    # insert the formatted data
cur.executemany("INSERT INTO ways_nodes(id, node_id,position) VALUES (?, ?, ?);", to_db)
# commit the changes
conn.commit()

conn.close()

In [164]:
cur.execute('''DROP TABLE ways_nodes''')

In [168]:
conn = sqlite3.connect(sqlite_file)
cur = conn.cursor()
cur.execute('''
  CREATE TABLE ways_tags(
id INTEGER ,
key TEXT,
value TEXT,
type TEXT)
''')
# commit the changes
conn.commit()

CSVFILE = "ways_tags.csv"

with open (CSVFILE, 'rb') as csvfile:
    # instead of using 'csv.DictReader' use `UnicodeDictReader`
    csv_reader = UnicodeDictReader(csvfile)

    to_db = [(i['id'], i['key'], i['value'],i['type']) for i in csv_reader]
    # insert the formatted data
cur.executemany("INSERT INTO ways_tags(id, key,value,type) VALUES (?, ?, ? ,? );", to_db)
# commit the changes
conn.commit()

conn.close()

In [179]:
conn = sqlite3.connect(sqlite_file)
cur = conn.cursor()


In [190]:

size of the file


number of chosen type of nodes, like cafes, shops etc.


### size of the file
```
 Date            Time       File size(MB)  File Name

2016/07/29  下午 04:45       514      hong-kong_china.osm
2016/07/31  下午 07:47       291      my_db.db
2016/07/31  下午 10:18       2.03     nodes.csv
2016/07/31  下午 10:18       67.3(KB) nodes_tags.csv
2016/07/31  下午 10:18       155(KB)  ways.csv
2016/07/31  下午 10:18       735(KB)  ways_nodes.csv
2016/07/31  下午 10:18       240(KB)  ways_tags.csv
  ```        

### number of unique  user 

In [200]:
## count the total of distinct user ID

cur.execute('''SELECT COUNT(DISTINCT(node_way.uid)) 
FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) node_way''')
print" Number of unique user:" 
MyPrettyPrinter().pprint(cur.fetchall())

 Number of unique user:
[(2824,)]


### number of nodes and ways

In [201]:
cur.execute('''SELECT COUNT(*) FROM nodes''')
print" Number of Nodes:" 
MyPrettyPrinter().pprint(cur.fetchall())


 Number of Nodes:
[(2574614,)]


In [202]:
cur.execute('''SELECT COUNT(*) FROM ways''')
print" Number of Ways:"
MyPrettyPrinter().pprint(cur.fetchall())

 Number of Ways:
[(265372,)]


### Top 10 city tags

In [205]:
cur.execute(''' SELECT tags.value, COUNT(*) as count 
FROM (SELECT * FROM nodes_tags UNION ALL 
      SELECT * FROM ways_tags) tags
WHERE tags.key LIKE '%city'
GROUP BY tags.value
ORDER BY count DESC
LIMIT 10''') ## SHOW FIRST top 10 tags

In [206]:
print "Top ten city tag :"
MyPrettyPrinter().pprint(cur.fetchall())

## among the top 10 tags, 3 of them are not in Hong Kong, (广东省深圳市, 50), (Zhuhai, 39), (深圳, 42).
## 

Top ten city tag :
[(香港 Hong Kong, 363),
 (屯門 Tuen Mun, 128),
 (荃灣 Tsuen Wan, 78),
 (Hong Kong, 54),
 (紅磡 Hung Hom, 54),
 (Sai Kung, 53),
 (广东省深圳市, 50),
 (深圳, 42),
 (Ta Kwu Ling, 41),
 (Zhuhai, 39)]


In [207]:
cur.execute(''' SELECT e.user, COUNT(*) as num
FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e
GROUP BY e.user
ORDER BY num DESC
LIMIT 10''')
print "Top 10 user:"
MyPrettyPrinter().pprint(cur.fetchall())


Top 10 user:
[(hlaw, 499316),
 (MarsmanRom, 245363),
 (Popolon, 160112),
 (sn0wblind, 120981),
 (katpatuka, 98086),
 (fsxy, 97258),
 (fdulezi, 84177),
 (KX675, 79145),
 (羊角忠实黑, 65681),
 (rainy3519446, 58215)]


In [228]:
cur.execute('''SELECT sum(num)  
FROM( SELECT  COUNT(*) as num
FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e
GROUP BY e.user
ORDER BY num DESC
LIMIT 10)''')
print "Sum of Top 10 users contributions:"
top_10=cur.fetchall()
MyPrettyPrinter().pprint(top_10)

##Add SELECT sum(num), which the previous top 10 query become a subquery

Sum of Top 10 users contributions:
[(1508334,)]


In [227]:
## Total number of contributions
cur.execute(''' SELECT count(*)
FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e

''')
print "Total Number of  contributions:"
total=cur.fetchall()
MyPrettyPrinter().pprint(total)


Total Number of  contributions:
[(2839986,)]


In [243]:
import numpy as np
print "Top 10 users contributions to the total contribution:" , np.array(top_10[0])*100.0/np.array(total[0]),"%"

## As the return type is tuple, I need to transfrom it to np type before divide them.

Top 10 users contributions to the total contribution: [ 53.11061393] %


In [246]:
cur.execute(''' SELECT value, COUNT(*) as num
FROM nodes_tags
WHERE key='food'
GROUP BY value
ORDER BY num DESC
LIMIT 10'''
            )
MyPrettyPrinter().pprint(cur.fetchall())


[(yes, 2)]


In [249]:
cur.execute(''' SELECT key, COUNT(*) as num
FROM nodes_tags
GROUP BY key
ORDER BY num DESC
LIMIT 20'''
            )
MyPrettyPrinter().pprint(cur.fetchall())
### The higest 20 count keys
### See what kind of key has the most tags, I see tag"tourism"  and "shop" which interest me, I will explore this 3 keys.


[(name, 28365),
 (highway, 19008),
 (power, 15790),
 (en, 11218),
 (zh, 10801),
 (amenity, 8616),
 (place, 6030),
 (crossing, 4929),
 (hkbus, 3644),
 (railway, 3550),
 (barrier, 3276),
 (operator, 2777),
 (ref, 2758),
 (tourism, 2590),
 (type, 2471),
 (kmb, 2139),
 (natural, 1970),
 (shop, 1865),
 (shelter, 1622),
 (street, 1531)]


In [256]:
cur.execute(''' SELECT value, COUNT(*) as num
FROM nodes_tags
WHERE key='tourism'
GROUP BY value
ORDER BY num DESC
LIMIT 20'''
)
MyPrettyPrinter().pprint(cur.fetchall())
### The 10 higest value with key=tourism
###  I can see Hong Kong have relatively few hostel compare to hotel in this query. There are 259 hotel and 44 hostel in the 
### database.

[(information, 1301),
 (attraction, 433),
 (hotel, 259),
 (viewpoint, 162),
 (guest_house, 141),
 (picnic_site, 96),
 (artwork, 71),
 (hostel, 44),
 (camp_site, 43),
 (museum, 31),
 (motel, 3),
 (zoo, 3),
 (chalet, 1),
 (gallery, 1),
 (theme_park, 1)]


In [257]:
cur.execute(''' SELECT value, COUNT(*) as num
FROM nodes_tags
WHERE key='amenity'
GROUP BY value
ORDER BY num DESC
LIMIT 10'''
)
MyPrettyPrinter().pprint(cur.fetchall())


[(toilets, 1115),
 (restaurant, 973),
 (shelter, 671),
 (bank, 598),
 (parking, 574),
 (bus_station, 573),
 (post_box, 572),
 (fast_food, 353),
 (cafe, 239),
 (fuel, 239)]


In [ ]:
## Again, it is funny that toliets have so many tags. 
## There are also so many banks tag that even more than bus station.

In [261]:
cur.execute(''' SELECT nodes.uid, COUNT(*) as num
FROM nodes
INNER JOIN nodes_tags
ON nodes.id=nodes_tags.id
WHERE value='toilets'
GROUP BY nodes.uid
ORDER BY num DESC
LIMIT 10'''
)
MyPrettyPrinter().pprint(cur.fetchall())
### Select user id and count who make the tag "toilets" by joining nodes_tags with nodes which share the same id
### It appear that 2 users "261189" and "169827" work quite hard to tag a lot of toilets!

[(261189, 317),
 (169827, 257),
 (2238851, 80),
 (230481, 36),
 (460264, 31),
 (31685, 26),
 (1665943, 22),
 (204590, 16),
 (1879520, 15),
 (2385962, 15)]


In [263]:
cur.execute(''' SELECT value, COUNT(*) as num
FROM nodes_tags
WHERE key='shop'
GROUP BY value
ORDER BY num DESC
LIMIT 20'''
)
MyPrettyPrinter().pprint(cur.fetchall())

[(convenience, 491),
 (supermarket, 377),
 (clothes, 140),
 (mall, 87),
 (bakery, 68),
 (books, 41),
 (chemist, 41),
 (jewelry, 40),
 (bicycle, 33),
 (kiosk, 33),
 (department_store, 29),
 (yes, 29),
 (electronics, 28),
 (car, 27),
 (hairdresser, 26),
 (gift, 22),
 (butcher, 20),
 (greengrocer, 20),
 (confectionery, 19),
 (mobile_phone, 19)]


In [ ]:
## There are many convenience tags, indeed Hong Kong have many convenience shops like 7-11, I believe the number is larger than 
## that. It surprise me supermarket have the second largest number of tag, I doubt it is the second common shops in Hong Kong.

## Conclusion
I think the Hong Kong dataset is incomplete. There are also some data belong to ShenZhen which is not inside Hong Kong. There are some users which have a great contribution to the dataset as I have seen. I am surprised that toliets have the most tags for amenity, I believe there are more restaurant than toilets in Hong Kong, there are still lots of thing to add into the dataset!

I have not tried to clean the chinese character for this project as I have spent some many time to input the data, it is painful that I can not output the correct data and put it into database. I believe the process should be similar but there are more work spend on dealing with unicode instead.


## Discussion

While the toilets tag surpurise me, I also think it may be a potential problem that certain users dominate the dataset, it will be better if more people participate and validate the data. It will be misleading if I say toilet is the most common amenity in Hong Kong and it certainly bring some problem to analysis. There are also many node with no tags while some of them have many tag. While it is a open dataset, is that possible to make a competition on some web like Kaggle to motivate more people participate in the dataset to help clean it?
I did not use OSM before, I am thinking that sometime there are many repeats tag for the same node or way, is that possible people can remove the tag freely? If not, is that possible people can give "likes" for a tag that the site will rank the top "likes" tag that people can figure out which tag is more reliable when there is more than one tag? It could be a problem if people can freely remove others tags, I think an option to "like" a tag and rank it may be the best practical solution.